In [1]:
import smtplib
import ssl
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import ipywidgets as widgets
from IPython.display import display, clear_output
import getpass
import os

def send_email(smtp_server, smtp_port, login_email, password, to_emails, subject, body, attachments):
    try:
        # Create a multipart message
        message = MIMEMultipart()
        message['From'] = login_email
        message['To'] = ', '.join(to_emails)
        message['Subject'] = subject

        # Attach the email body
        message.attach(MIMEText(body, 'plain'))

        # Attach files if any
        for file_path in attachments:
            try:
                with open(file_path, 'rb') as f:
                    part = MIMEApplication(f.read(), Name=os.path.basename(file_path))
                part['Content-Disposition'] = f'attachment; filename="{os.path.basename(file_path)}"'
                message.attach(part)
            except Exception as e:
                print(f"Could not attach {file_path}: {e}")

        # Create a secure SSL context
        context = ssl.create_default_context()

        # Connect to the SMTP server and send the email
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls(context=context)  # Secure the connection
            server.login(login_email, password)
            server.sendmail(login_email, to_emails, message.as_string())

        print("Email sent successfully!")

    except Exception as e:
        print(f"An error occurred: {e}")

# Define widgets for user input
smtp_server = widgets.Text(
    value='smtp.gmail.com',
    placeholder='Enter SMTP server',
    description='SMTP Server:',
    disabled=False
)

smtp_port = widgets.IntText(
    value=587,
    description='SMTP Port:',
    disabled=False
)

login_email = widgets.Text(
    value='',
    placeholder='Enter your email',
    description='Your Email:',
    disabled=False
)

password = widgets.Password(
    value='',
    placeholder='Enter your password',
    description='Password:',
    disabled=False
)

to_emails = widgets.Text(
    value='',
    placeholder='Enter recipient emails separated by commas',
    description='To Emails:',
    disabled=False
)

subject = widgets.Text(
    value='',
    placeholder='Enter email subject',
    description='Subject:',
    disabled=False
)

body = widgets.Textarea(
    value='',
    placeholder='Enter email body',
    description='Body:',
    disabled=False
)

attachments = widgets.Text(
    value='',
    placeholder='Enter file paths separated by commas (optional)',
    description='Attachments:',
    disabled=False
)

send_button = widgets.Button(
    description='Send Email',
    button_style='success',  # Options: 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to send email',
    icon='envelope'  # (FontAwesome names without the `fa-` prefix)
)

def on_send_button_clicked(b):
    with output:
        clear_output()  # Clear previous output
        # Retrieve values from widgets
        smtp = smtp_server.value
        port = smtp_port.value
        sender_email = login_email.value
        sender_password = password.value
        recipients = [email.strip() for email in to_emails.value.split(',')]
        email_subject = subject.value
        email_body = body.value
        attachment_paths = [path.strip() for path in attachments.value.split(',')] if attachments.value else []

        # Basic validation
        if not sender_email or not sender_password or not recipients:
            print("Please fill in all required fields: Your Email, Password, and To Emails.")
            return

        # Call the send_email function
        send_email(
            smtp_server=smtp,
            smtp_port=port,
            login_email=sender_email,
            password=sender_password,
            to_emails=recipients,
            subject=email_subject,
            body=email_body,
            attachments=attachment_paths
        )

# Link the button click event to the callback function
send_button.on_click(on_send_button_clicked)

# Output widget to display messages
output = widgets.Output()

# Arrange widgets in a vertical layout
form_items = [
    smtp_server,
    smtp_port,
    login_email,
    password,
    to_emails,
    subject,
    body,
    attachments,
    send_button,
    output
]

# Display the form
display(widgets.VBox(form_items))
